In [7]:
!pip install langchain_openai langchain_community langchain_openai tavily-python -q
!pip install openai -q

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

··········


In [4]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


In [21]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from tavily import TavilyClient
import os

# 确保设置了 Tavily API 密钥
tavily_api_key = os.getenv("TAVILY_API_KEY")
if not tavily_api_key:
    raise ValueError("请设置 TAVILY_API_KEY 环境变量")

# 创建 Tavily 客户端
tavily_client = TavilyClient(api_key=tavily_api_key)

# 创建一个使用 Tavily API 的搜索函数
def tavily_search(query: str) -> str:
    try:
        results = tavily_client.search(query)
        # 返回第一个结果的摘要，如果有的话
        if results and len(results) > 0:
            return results
        else:
            return "No results found."
    except Exception as e:
        return f"Error performing search: {str(e)}"

# 定义一个简单的计算工具函数
def calculate_square(number_str: str) -> str:
    """
    计算一个整数的平方。
    """
    try:
        number = int(number_str)
        return f"The square of {number} is {number ** 2}"
    except ValueError:
        return f"Error: '{number_str}' is not a valid integer."

# 将 Tavily 搜索工具和计算工具包装成 LangChain 的 Tool
tools = [
    Tool(
        name="Search",
        func=tavily_search,
        description="Useful for when you need to answer questions by searching the web."
    ),
    Tool(
        name="SquareCalculator",
        func=calculate_square,
        description="Calculates the square of a given number. Input should be a string representing an integer."
    )
]

# 初始化 OpenAI 模型
llm = OpenAI(temperature=0)

# 初始化 ReAct 代理
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 使用代理处理请求
response = agent.run("What is the square of 15? Also, what is the capital of France?")

# 输出结果
print(response)



> Entering new AgentExecutor chain...
 I should use the SquareCalculator for the first question and Search for the second question.
Action: SquareCalculator
Action Input: "15"
Observation: The square of 15 is 225
Thought: I should now use Search to find the capital of France.
Action: Search
Action Input: "capital of France"
Observation: {'query': 'capital of France', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Capital of France - Simple English Wikipedia, the free encyclopedia', 'url': 'https://simple.wikipedia.org/wiki/Capital_of_France', 'content': 'Learn about the history and current status of the capital of France, which is Paris. Find out which other cities have been the national capital in different periods of time, from ancient to modern.', 'score': 0.9972638, 'raw_content': None}, {'title': 'List of capitals of France - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/List_of_capitals_of_France', 'content': 'Learn about the historical an